In this exercise you will use Python's hplib library to model heat pump performance across different climates and building insulation levels with the ultimate goal of comparing air and ground source heat pumps.

Note you do not need to be a programmer to do this exercise! code blocks are ready to run, and certain key variables that you can change are clearly marked in the code. simply change values after the '=' sign and press play!

This exercise will lay the groundwork for economic modelling that will happen later in the course. For now the goal is to understand how key energetic metrics can be found, which will later be used as inputs to derive price comparisons

In [11]:
# Clone the hplib repository
!git clone https://github.com/RE-Lab-Projects/hplib.git

# Navigate to the cloned directory to make it accessible
import sys
sys.path.append('/content/hplib')


Cloning into 'hplib'...
remote: Enumerating objects: 14630, done.
remote: Counting objects: 100% (14630/14630), done.
remote: Compressing objects: 100% (4253/4253), done.
remote: Total 14630 (delta 10405), reused 14529 (delta 10308), pack-reused 0 (from 0)
Receiving objects: 100% (14630/14630), 454.59 MiB | 25.18 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.
Updating files: 100% (2667/2667), done.


In [26]:
import os
print(os.getcwd())
os.chdir(r'../hplib')
import hplib as hpl

import importlib
importlib.reload(hplib)

import hplib_database as db
import pandas as pd
import warnings
import numpy
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

/content/hplib/hplib


In [27]:
database = hpl.load_database()
database

,Manufacturer,Model,Titel,Date,Type,Subtype,Group,Rated Power low T [kW],Rated Power medium T [kW],Refrigerant,...,p1_EER [-],p2_EER [-],p3_EER [-],p4_EER [-],MAPE P_th,MAPE P_el,MAPE COP,MAPE Pdc,MAPE P_el_c,MAPE EER
0,Acond a.s.,Acond Aconomis N,Acond Aconomis N,08.07.2024,Outdoor Air/Water,Regulated,1.0,7.65,7.5,R290,...,NaN,NaN,NaN,NaN,13.264637,11.112413,6.552139,NaN,NaN,NaN
1,Acond a.s.,Acond Aconomis S,Acond Aconomis S,08.07.2024,Outdoor Air/Water,Regulated,1.0,4.00,4.0,R290,...,NaN,NaN,NaN,NaN,13.353078,11.885422,5.921082,NaN,NaN,NaN
2,Acond a.s.,Acond Grandis-L,Acond Grandis-L16,31.01.2024,Outdoor Air/Water,Regulated,1.0,15.50,16.0,R290,...,NaN,NaN,NaN,NaN,14.109089,7.289588,12.571346,NaN,NaN,NaN
3,Acond a.s.,Acond Grandis-L,Acond Grandis-L18,31.01.2024,Outdoor Air/Water,Regulated,1.0,18.80,18.0,R290,...,NaN,NaN,NaN,NaN,28.626242,21.773224,14.423571,NaN,NaN,NaN
4,Acond a.s.,Acond Grandis-L,Acond Grandis-L21,31.01.2024,Outdoor Air/Water,Regulated,1.0,21.00,22.0,R290,...,NaN,NaN,NaN,NaN,42.116176,31.044366,17.378241,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9948,Generic,Generic,Generic_bottom,NaN,Outdoor Air/Water,On-Off,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0
9949,Generic,Generic,Generic_top,NaN,Brine/Water,On-Off,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0
9950,Generic,Generic,Generic_average,NaN,Brine/Water,On-Off,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0
9951,Generic,Generic,Generic_bottom,NaN,Brine/Water,On-Off,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0


as a first step, lets see if we can get heating demand estimates on a monthly level. There are many ways to do this and a lot of resources online. You can also just look up numbers.

However, we will do this by just assuming there is a thermal energy loss per m2 per square meter is a linear function of the temperature difference between inside and outside and some insulation factor.

Monthly Heat demand (kWh/m2 * month) = (Tout - Tin) * Insulation Factor

In [28]:
parameters = hpl.get_parameters('i-SHWAK V4 06')
parameters

,Manufacturer,Model,Titel,Date,Type,Subtype,Group,Rated Power low T [kW],Rated Power medium T [kW],Refrigerant,...,p1_EER [-],p2_EER [-],p3_EER [-],p4_EER [-],MAPE P_th,MAPE P_el,MAPE COP,MAPE Pdc,MAPE P_el_c,MAPE EER
76,Advantix S.p.A.,i-SHWAK V4 06,i-SHWAK V4 06,26.05.2020,Outdoor Air/Water,Regulated,1.0,5.0,4.0,R410A,...,128.272732,0.185413,7.252562,-128.438566,24.897666,22.607658,9.244852,8.246285,13.638634,7.857755


In [ ]:
# this function just returns the heat load (in kWh/m2) for a specific amount of time
# If we enter 720 hours (one month) we will get the average value for that month

def heating_demand(insulation_level, indoor_temp, outdoor_temp, area, hours):
    # Base heat loss rate per m² at different insulation levels (in W/m²·K)
    heat_loss_rates = {'low': 2.0, 'medium': 1.0, 'high': 0.5}
    heat_loss_rate = heat_loss_rates[insulation_level]

    # Calculate thermal load based on temperature difference
    thermal_load = heat_loss_rate * area * (indoor_temp - outdoor_temp)

    # Convert to kWh and multiply by hours of operation
    return thermal_load * hours / 1000

In [ ]:
# Define insulation levels (e.g., high, medium, low insulation) in kWh/m²/month
insulation_levels = {
    'high': 5,  # High insulation (e.g., passive house)
    'medium': 15,  # Medium insulation (e.g., modern building)
    'low': 30   # Low insulation (e.g., older building)
}

# Define average monthly temperatures for different climates (Celsius). This list is represents average temper
# over a year starting in January
climates = {
    'mild': [10, 8, 5, 7, 12, 15, 18, 18, 14, 10, 7, 5],  # mild, year-round
    'cold': [-5, -3, 0, 3, 8, 12, 15, 13, 9, 5, 0, -2],   # colder winter, moderate summer
    'very_cold': [-10, -8, -5, -2, 5, 10, 12, 10, 6, 2, -5, -8]  # very cold winter, short summer
}


now we can play with some values here and compare to something from literature.
try changing the insulation levels and the average monthly temperatures in the code block above and hitting play (you have to hit play on the code block abova and below to refresh the numbers)

In [ ]:
def calculate_cop(heat_pump_type, outdoor_temp):
    """
    Calculate COP for a geothermal or air-source heat pump based on outdoor temperature.
    `heat_pump_type`: 'geothermal' or 'air-source'
    `outdoor_temp`: Current outdoor temperature in Celsius
    """
    if heat_pump_type == 'geothermal':
        cop = hplib.get_parameters(model='Generic_top', group_id=1, t_in=-7, t_out=40, p_th=10000)["COP"] # Replace with actual HPLib function for geothermal
    elif heat_pump_type == 'air-source':
        cop = hplib.air_source_COP(outdoor_temp)  # Replace with actual HPLib function for air-source
    return cop


In [ ]:
# Initialize results dictionary
results = []

# Iterate over each insulation level and climate
for insulation, demand in insulation_levels.items():
    for climate, temps in climates.items():
        monthly_demand = demand  # Demand in kWh/m²/month based on insulation level
        for month, temp in enumerate(temps):
            # Calculate COPs for both types of heat pumps
            cop_geo = calculate_cop('geothermal', temp)
            cop_air = calculate_cop('air-source', temp)

            # Calculate energy consumption based on demand and COP
            energy_geo = monthly_demand / cop_geo
            energy_air = monthly_demand / cop_air

            # Append results
            results.append({
                'insulation': insulation,
                'climate': climate,
                'month': month + 1,
                'temp': temp,
                'cop_geo': cop_geo,
                'cop_air': cop_air,
                'energy_geo': energy_geo,
                'energy_air': energy_air
            })

# Convert results to DataFrame for analysis and visualization
df = pd.DataFrame(results)


AttributeError: module 'hplib' has no attribute 'get_parameters'

In [ ]:
# Plot energy consumption for each climate and insulation level
for insulation in insulation_levels.keys():
    for climate in climates.keys():
        # Filter data for the given insulation level and climate
        data = df[(df['insulation'] == insulation) & (df['climate'] == climate)]

        # Plot monthly energy consumption
        plt.figure(figsize=(10, 6))
        plt.plot(data['month'], data['energy_geo'], label='Geothermal', marker='o')
        plt.plot(data['month'], data['energy_air'], label='Air-Source', marker='o')

        # Add plot details
        plt.title(f'Energy Consumption ({insulation.capitalize()} Insulation, {climate.capitalize()} Climate)')
        plt.xlabel('Month')
        plt.ylabel('Energy Consumption (kWh/m²)')
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:
print(dir(hplib))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_version']
